## Exercício sobre pytorch

Utilize o dataset 'datasetCarros.csv'.<br>
Usando Pytorch, construa uma rede neural para prever a feature 'PrecoVenda'.<br>

Use uma rede neural feed forward com duas camadas escondidas, com 50 neurônios cada.<br>
Use o critério de perda MSELoss, otimizador Adam e learning rate = 0.001. Considere 10000 épocas.

In [57]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

# Define a classe da rede neural
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Carregando o conjunto de dados
df = pd.read_csv('datasetCarros.csv', dtype={
    'Nome': str,
    'Ano': int,
    'PrecoVenda': float,
    'PrecoAtual': float,
    'KmRodado': int,
    'TipoCombustivel': str,
    'Trasmissao': str,
    'Owner': int
})
# remove colunas string
df = df.drop(columns=['Nome', 'TipoCombustivel', 'Trasmissao'])

# Separando as features e o target
X = df.drop('PrecoVenda', axis=1).values
y = df['PrecoVenda'].values

# Normalizando as features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)

X = (X - X_mean) / X_std

# Convertendo os dados para tensores PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)
y_tensor = y_tensor.unsqueeze(1)

# Definindo uma classe Dataset personalizada
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

# Criando os DataLoaders para o conjunto de treinamento e teste
dataset = CustomDataset(X_tensor, y_tensor)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Instanciando a rede neural, o critério de perda e o otimizador
net = Net(input_size=4, hidden_size=50, output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Treinamento da rede neural
for epoch in range(10000):
    running_loss = 0.0
    for data in train_loader:
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if epoch % 1000 == 999:
        print(f'[{epoch + 1}] loss: {running_loss / 1000:.3f}')
        running_loss = 0.0

print('Finished Training')

# Teste da rede neural
with torch.no_grad():
    total_loss = 0
    for inputs, labels in test_loader:
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        print(total_loss)


[1000] loss: 0.002
[2000] loss: 0.002
[3000] loss: 0.001
[4000] loss: 0.001
[5000] loss: 0.001
[6000] loss: 0.001
[7000] loss: 0.001
[8000] loss: 0.001
[9000] loss: 0.001
[10000] loss: 0.001
Finished Training
0
